In [1]:
import sys
from pathlib import Path
parent_dir = Path('.').resolve().parent
sys.path.insert(0, str(parent_dir))

import numpy as np
import pandas as pd
import math
import statistics
import random
import importlib

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

import math_expressions as mexpr
importlib.reload(mexpr)

import methods
importlib.reload(methods)

<module 'methods' from '/Users/preetkaria/Desktop/RA/time_to_leave/methods.py'>

In [2]:
df = pd.read_csv('../data/gen_data.csv')

In [3]:
df.head()

,alpha,beta,h,c,N,n,mean_n,std_n,alpha_hat,beta_hat,u,u_star,u_star_hat,z,optimal_cost,actual_cost,intervals_str
0,2,1,0.666667,25,11,5,1.873082,1.461553,2,1.140439,17.568911,14.624458,17.011105,0.859701,1.962969,0.371871,4.075370888878726_1.1961676454020844_0.9112248...
1,2,4,0.133333,10,11,5,5.342704,2.689391,4,1.353776,84.592079,65.510277,45.236862,1.448161,2.544240,5.247362,4.13826760376251_8.478514581903736_1.660076247...
2,2,4,0.133333,10,21,5,7.936178,4.567746,3,2.629011,154.519799,143.636724,142.921733,1.005003,1.451077,1.546409,11.819341605892342_13.645102878552056_2.912788...
3,3,4,0.666667,20,14,5,7.909362,3.776084,4,1.802776,168.547142,169.554063,91.491101,1.853230,20.000000,51.370694,9.435224942661229_6.3361516932619235_13.075884...
4,3,2,0.133333,30,37,5,3.944788,3.060492,2,2.374427,226.232805,185.745572,140.319334,1.323735,5.398298,11.455129,6.0115182856632785_2.589541520901462_2.0345337...


In [4]:
X = df[['N', 'n', 'h', 'c', 'mean_n', 'std_n', 'alpha_hat', 'beta_hat', 'u_star_hat']]
y = df['z']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=5)


In [5]:
model = LinearRegression()

model.fit(X_train, y_train)
display(model.score(X_train, y_train))
display(model.coef_)

0.11711962051816482

array([ 4.63676120e-03,  3.87439646e-16,  1.15044279e-01, -2.67202903e-04,
        1.44713624e-02, -4.10124543e-02, -2.07099725e-02, -1.29290989e-02,
       -3.00113301e-04])

In [6]:
test_output = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=['predicted_z'])
test_df = test_output.merge(df, left_index=True, right_index=True)
test_df['predicted_u_star'] = test_df['predicted_z'] * test_df['u_star_hat']
test_df['actual_cost'] = test_df.apply(lambda row: methods.cal_cost(row['c'], row['h'], row['u'], row['predicted_u_star']), axis=1)
test_df.head()

,predicted_z,alpha,beta,h,c,N,n,mean_n,std_n,alpha_hat,beta_hat,u,u_star,u_star_hat,z,optimal_cost,actual_cost,intervals_str,predicted_u_star
148,1.080564,3,2,0.066667,25,30,5,6.677872,4.674266,2,3.271815,194.128457,142.184100,149.130568,0.953420,3.462957,2.198885,2.036644172972612_6.825692918927528_6.39395201...,161.145182
3087,0.950356,2,3,0.666667,25,15,5,11.074079,8.221451,2,6.103645,111.146834,77.565829,186.485064,0.415936,22.387336,25.000000,2.9270201503321944_12.091257917815105_4.570117...,177.227256
3607,0.961168,6,3,0.133333,20,4,5,24.623126,7.924499,10,2.550354,64.788620,47.282698,75.445658,0.626712,2.334123,20.000000,21.280110638920178_36.10008334878621_14.386531...,72.515978
3672,1.061157,4,1,0.066667,30,14,5,4.149473,1.530784,7,0.564722,56.985189,38.332324,41.390085,0.926123,1.243524,0.870919,2.016362555395852_5.891464155784151_3.37246720...,43.921399
406,1.048651,2,4,0.133333,10,35,5,10.487395,4.439126,6,1.879002,277.754779,256.579860,371.432899,0.690784,2.823323,10.000000,8.513765873359434_11.159780639269837_6.0548261...,389.503308


In [7]:
methods.plot_plotly(data=test_df['actual_cost'], data_label='Actual Cost')
methods.plot_plotly(data=test_df['optimal_cost'], data_label='Optimal Cost')
methods.multi_plot_plotly(data=[test_df['optimal_cost'], test_df['actual_cost']], data_label=['Optimal Cost', 'Actual Cost'])
